In [67]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px


In [68]:
df= pd.read_csv(r"D:\Piyush Domadiya\Day11\datasheet.csv")


In [69]:
df = df[df['Day'] != 'Period Total'].reset_index(drop=True)

# Convert Day column to datetime
df['Day'] = pd.to_datetime(df['Day'], format='%a %d-%b', errors='coerce')
df = df.dropna(subset=['Day'])

# Function to parse 'HH:MM' string to total hours (float) or timedelta
def parse_hours(t):
    if pd.isna(t) or t == '--:--':
        return 0.0
    try:
        h, m = map(int, t.split(':'))
        return h + m/60.0
    except:
        return 0.0

# Apply parsing
df['Total Hours Float'] = df['Total Ex Idle Hours'].apply(parse_hours)

# Late Punch Logic (Assuming > 10:00 AM is late)
def check_late(t):
    if pd.isna(t) or t == '--:--':
        return False
    try:
        # Compare string directly if format is HH:MM
        return t > '10:00'
    except:
        return False

df['Late Punch'] = df['Punch-in'].apply(check_late)

# Overtime Logic (> 8.0 hours)
df['Overtime'] = df['Total Hours Float'] > 8.0

# Half Day Logic (< 4.5 hours)
df['Half Day'] = (df['Total Hours Float'] <= 4.5) & (df['Total Hours Float'] > 0)

# Leave Day Logic (0 hours = leave/absent)
df['Leave Day'] = (
    (df['Total Hours Float'] == 0.0) &
    (df['Day'].dt.dayofweek < 5)
)
# Day Type for color coding in bar chart
def categorize_day(hours):
    if hours == 0:
        return 'Leave'
    elif hours < 4.5:
        return 'Half Day'
    elif hours > 8.10:
        return 'Overtime'
    else:
        return 'Regular'

df['Day Type'] = df['Total Hours Float'].apply(categorize_day)

# Formatted day label for x-axis (e.g., 'S-2-D')
df['Day Label'] = (
    df['Day'].dt.strftime('%a').str[0] + "-" +
    df['Day'].dt.strftime('%d') + "-" +
    df['Day'].dt.strftime('%b').str[0]
)


In [70]:
app = dash.Dash(__name__)

In [71]:
# Bar Chart with color coding and formatted x-axis
bar_fig = px.bar(
    df,
    x="Day Label",
    y="Total Hours Float",
    color="Day Type",
    color_discrete_map={
        'Regular': '#2E86AB',   # Blue
        'Half Day': '#F77F00',  # Orange
        'Overtime': '#06A77D',  # Green
        'Leave': '#D62828'      # Red
    },
    title="Total Work Hours Per Day (Color Coded)"
)

# Maintain proper date order
bar_fig.update_layout(
    xaxis={
        'categoryorder': 'array',
        'categoryarray': df.sort_values("Day")["Day Label"]
    }
)

# X-axis label name
bar_fig.update_xaxes(title_text="Day (D-Date-M)")

bar_fig.update_layout(
    width=800,
    xaxis=dict(
        tickmode='array',
        tickvals=df.sort_values("Day")["Day Label"],
        ticktext=df.sort_values("Day")["Day Label"]
    )
)

bar_fig.update_xaxes(tickangle=60)



In [72]:
# Line Chart with Date-wise dots and weekend gap

line_fig = px.line(
    df,
    x='Day',
    y='Total Hours Float',
    title='Work Hours Trend',
    markers=True
)

# Break line on missing / zero values (if any)
line_fig.update_traces(
    connectgaps=False,
    marker=dict(size=8),     # Dot size
    line=dict(width=3)       # Line thickness
)

# Improve x-axis formatting
line_fig.update_xaxes(
    tickformat="%d-%b",      # Format like 02-Dec
    tickangle=45,
    title_text="Date"
)
# Improve layout
line_fig.update_layout(
    height=500,
    width=800,
    yaxis_title="Work Hours"
)
# Maintain proper date order
bar_fig.update_layout(
    xaxis={
        'categoryorder': 'array',
        'categoryarray': df.sort_values("Day")["Day Label"]
    }
)




In [73]:
# Pie Charts
overtime_count = df['Overtime'].sum()
normal_count = len(df) - overtime_count

overtime_fig = px.pie(
    names=['Overtime', 'Normal'],
    values=[overtime_count, normal_count],
    title='Overtime Days'
)


In [74]:
# Improved Late Punch Logic (Time comparison properly)

late_fig = px.pie(
    names=["Late", "On Time"],
    values=[df["Late Punch"].sum(),
            len(df) - df["Late Punch"].sum()],
    title="Late Punch"
)



In [75]:
halfday_fig = px.pie(
    names=["Half Day", "Full Day"],
    values=[df["Half Day"].sum(),
            len(df) - df["Half Day"].sum()],
    title="Half Day"
)

In [76]:
leave_fig = px.pie(
    names=["Leave", "Present"],
    values=[df["Leave Day"].sum(),
            len(df) - df["Leave Day"].sum()],
    title="Leave Days"
)
def check_late(t):
    if pd.isna(t) or t == '--:--':
        return False
    try:
        time_obj = pd.to_datetime(t, format='%H:%M').time()
        return time_obj > pd.to_datetime('10:10', format='%H:%M').time()
    except:
        return False

df['Late Punch'] = df['Punch-in'].apply(check_late)
late_fig = px.pie(
    names=['Late', 'On Time'],
    values=[df['Late Punch'].sum(),
            len(df) - df['Late Punch'].sum()],
    title='Late Punch'
)

In [77]:
# Layout

app.layout = html.Div([
    html.H1(
        "Attendance and Work Hours Dashboard",
    style={
            'textAlign': 'center',
            'color': 'lightgreen',
            'fontFamily': 'Arial',
            'marginBottom': '20px'
        }
    ),
    dcc.Graph(figure=bar_fig,
    style = { 
        'width':'700px',
        'justifyContent': 'center'
    }
    ),
    # FIRST ROW (2 pie charts side by side)
    html.Div([
        dcc.Graph(figure=overtime_fig, style={'width': '48%'}),
        dcc.Graph(figure=late_fig, style={'width': '48%'})
    ], style={
        'display': 'flex',
        
    }),

    # SECOND ROW (2 pie charts side by side)
    html.Div([
        dcc.Graph(figure=halfday_fig, style={'width': '48%'}),
        dcc.Graph(figure=leave_fig, style={'width': '48%'})
    ], style={
        'display': 'flex',
        'justifyContent': 'space-between'
    }),

    dcc.Graph(figure=line_fig)

])


In [78]:
# Run App

app.run(debug=True)
